# 🧠 Träning & ONNX Export - Förstå vad som händer

**Mål**: Förstå hur träning fungerar och experimentera med olika inställningar.

I detta notebook kommer vi att:
- Förstå vad FakeData är och varför vi använder det
- Se hur dataset-pipeline → modell → loss/accuracy fungerar
- Experimentera med olika hyperparametrar
- Förstå varför vi exporterar till ONNX

> **💡 Tips**: Kör cellerna i ordning och läs förklaringarna. Experimentera gärna med värdena!


## 🤔 Vad är FakeData och varför använder vi det?

**FakeData** är syntetiska bilder som PyTorch genererar automatiskt. Det är perfekt för:
- **Snabb prototyping** - ingen nedladdning av stora dataset
- **Reproducerbarhet** - samma data varje gång
- **Undervisning** - fokus på algoritmer, inte datahantering

<details>
<summary>🔍 Klicka för att se vad FakeData innehåller</summary>

```python
# FakeData genererar:
# - Slumpmässiga RGB-bilder (64x64 pixlar)
# - Slumpmässiga klasser (0, 1, 2, ...)
# - Samma struktur som riktiga bilddataset
```

</details>


In [ ]:
# Låt oss skapa en liten FakeData för att se vad den innehåller
import torch
from torchvision import datasets
import matplotlib.pyplot as plt

# Skapa FakeData med 2 klasser
fake_data = datasets.FakeData(size=10, num_classes=2, transform=None)

# Visa första bilden
image, label = fake_data[0]
print(f"Bildstorlek: {image.size}")
print(f"Klass: {label}")
print(f"Pixelvärden: {image.getextrema()}")

# Visa bilden
plt.figure(figsize=(6, 4))
plt.imshow(image)
plt.title(f"FakeData - Klass {label}")
plt.axis('off')
plt.show()


## 🎯 Experimentera med Träning

Nu ska vi träna en modell och se hur olika inställningar påverkar resultatet.

**Hyperparametrar att experimentera med**:
- `epochs` - antal genomgångar av datasetet
- `batch_size` - antal bilder per träningssteg
- `--no-pretrained` - börja från noll vs förtränade vikter


In [ ]:
# Experiment 1: Snabb träning (1 epoch, ingen pretrained)
print("🧪 Experiment 1: Snabb träning")
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 128 --output-dir ./models_exp1


In [ ]:
# Visa träningsresultat från Experiment 1
import json
import os

if os.path.exists("./models_exp1/training_info.json"):
    with open("./models_exp1/training_info.json", "r") as f:
        info = json.load(f)
    
    print("📊 Träningsresultat (Experiment 1):")
    print(f"Final accuracy: {info.get('final_accuracy', 'N/A'):.3f}")
    print(f"Final loss: {info.get('final_loss', 'N/A'):.3f}")
    print(f"Epochs: {info.get('epochs', 'N/A')}")
    print(f"Batch size: {info.get('batch_size', 'N/A')}")
else:
    print("❌ Träningsinfo saknas")


## 🤔 Reflektionsfrågor

<details>
<summary>💭 Vad händer med överfitting när du höjer epochs?</summary>

**Svar**: Med fler epochs kan modellen lära sig träningsdata för bra och dåligt generalisera till nya data. Detta kallas överfitting.

**Experiment**: Kör samma träning men med `--epochs 5` och jämför accuracy på tränings- vs valideringsdata.

</details>

<details>
<summary>💭 Varför exporterar vi till ONNX (för Pi/edge)?</summary>

**Svar**: ONNX är ett standardformat som fungerar på många plattformar (CPU, GPU, mobil, edge). Det gör modellen portabel och optimerad för inference.

**Fördelar**:
- Snabbare inference än PyTorch
- Mindre minnesanvändning
- Fungerar på Raspberry Pi
- Stöd för kvantisering (INT8)

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Träna en modell med andra inställningar och jämför resultaten.

**Förslag**:
- Öka epochs till 3-5
- Ändra batch_size till 64 eller 256
- Testa med och utan `--no-pretrained`

**Kod att modifiera**:
```python
# Ändra dessa värden:
EPOCHS = 3
BATCH_SIZE = 64
USE_PRETRAINED = False  # True för förtränade vikter

!python -m piedge_edukit.train --fakedata --epochs {EPOCHS} --batch-size {BATCH_SIZE} --output-dir ./models_myexp
```


In [ ]:
# TODO: Implementera ditt experiment här
# Ändra värdena nedan och kör träningen

EPOCHS = 3
BATCH_SIZE = 64
USE_PRETRAINED = False

print(f"🧪 Mitt experiment: epochs={EPOCHS}, batch_size={BATCH_SIZE}, pretrained={USE_PRETRAINED}")

# TODO: Kör träningen med dina inställningar
# !python -m piedge_edukit.train --fakedata --epochs {EPOCHS} --batch-size {BATCH_SIZE} --output-dir ./models_myexp


## 🎉 Sammanfattning

Du har nu lärt dig:
- Vad FakeData är och varför vi använder det
- Hur träning fungerar med olika hyperparametrar
- Varför ONNX-export är viktigt för edge deployment

**Nästa steg**: Gå till `02_latency_benchmark.ipynb` för att förstå hur vi mäter modellens prestanda.

**Viktiga begrepp**:
- **Epochs**: Antal genomgångar av datasetet
- **Batch size**: Antal bilder per träningssteg
- **Pretrained weights**: Förtränade vikter från ImageNet
- **ONNX**: Standardformat för edge deployment
